This notebook script will print out the per channel accuracy of a multi-channel architecture model

In [1]:
import numpy as np
import pandas as pd

In [2]:
def calculate(tolerance, path, wavelengths):
  results = np.load(path)
  num_channels = len(wavelengths)
  primary_metrics = [0] * num_channels
  for c in range(num_channels):
    gt = results[:, c, 0]
    pred = results[:, c, 1]

    diff = np.abs(pred - gt)

    batch_size = diff.shape[0]
    n_elems = batch_size

    primary_metrics[c] = np.sum(diff < tolerance, dtype=np.float32) / n_elems
    
  return np.insert(np.array(primary_metrics), 0, tolerance)

In [3]:
# Multi-channel model results.
tolerances = [0.1, 0.05, 0.01]
wavelengths = ["0094",
               "0131",
               "0171",
               "0193",
               "0211",
               "0304",
               "0335"]
results = np.zeros((len(tolerances), len(wavelengths) + 1))

for idx, tolerance in enumerate(tolerances):
  results[idx] = calculate(tolerance=tolerance,
                           path='/gpfs/gpfs_gl4_16mb/b9p111/fdl_sw/experiments_results/brad_exp_25/5000_brad_exp_25_test_predictions.npy',
                           wavelengths=wavelengths)
columns = ['Tolerance']
columns.extend(wavelengths.copy())
pd.DataFrame(results, columns=columns)

,Tolerance,0094,0131,0171,0193,0211,0304,0335
0,0.10,1.000000,0.998553,1.000000,1.000000,1.000000,0.998553,0.998553
1,0.05,0.994211,0.994211,0.997106,0.988423,0.998553,0.927641,0.998553
2,0.01,0.539797,0.571635,0.500724,0.623734,0.619392,0.390738,0.628075


In [9]:
# Model that only takes one wavelength to process results.
tolerances = [0.1, 0.05, 0.01]
wavelengths = ["0094",
               "0131",
               "0171",
               "0193",
               "0211",
               "0304",
               "0335"]
results = np.zeros((len(tolerances), len(wavelengths) + 1))

for tolerance_idx, tolerance in enumerate(tolerances):
  results[tolerance_idx] = tolerance
  for wavelength_idx, wavelength in enumerate(wavelengths):
    path = '/gpfs/gpfs_gl4_16mb/b9p111/b9p111al/experiments_results/brad_exp_24_{}/0050_brad_exp_24_{}_test_predictions.npy'.format(wavelength, wavelength)
    results[tolerance_idx, wavelength_idx + 1] = calculate(tolerance=tolerance,
                                                           path=path,
                                                           wavelengths=[wavelength])[1]

columns = ['Tolerance']
columns.extend(wavelengths.copy())
pd.DataFrame(results, columns=columns)

,Tolerance,0094,0131,0171,0193,0211,0304,0335
0,0.10,0.992785,1.000000,1.000000,0.982659,0.878788,0.979769,0.953757
1,0.05,0.862915,0.946532,0.981187,0.800578,0.636364,0.885838,0.755780
2,0.01,0.300144,0.371387,0.474674,0.332370,0.227994,0.410405,0.235549
